In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from pyod.models.gmm import GMM
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# from pyod.models.iforest import IForest
# from pyod.models.gmm import GMM
# from sklearn.svm import OneClassSVM
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense
# import numpy as np
# from sklearn.model_selection import train_test_split


In [ ]:
def clean_and_prepare(df):
    """
    Cleans and prepares the dataframe:
    - Ensures datetime index
    - Handles missing values without affecting outlier detection
    - Creates time-based features
    - Drops non-numeric data
    """

    # Ensure datetime index
    if not isinstance(df.index, pd.DatetimeIndex):
        df["dt_time"] = pd.to_datetime(df["dt_time"])
        df = df.set_index("dt_time")

    # Remove non-relevant columns
    if "deviceid" in df.columns:
        df.drop(columns=["deviceid"], inplace=True)

    # Interpolate missing values using time-based method
    df.interpolate(method="time", inplace=True)

    # Create time-based features
    df["hour"] = df.index.hour
    df["sin_hour"] = np.sin(2 * np.pi * df["hour"] / 24)
    df["cos_hour"] = np.cos(2 * np.pi * df["hour"] / 24)
    # df["day_of_week"] = df.index.dayofweek
    # df["sin_day"] = np.sin(2 * np.pi * df["day_of_week"] / 7)
    # df["cos_day"] = np.cos(2 * np.pi * df["day_of_week"] / 7)

    # Lag Features (Avoid NaNs)
    df["pm2.5_lag1"] = df["pm2.5cnc"].shift(1).bfill()
    df["pm2.5_lag2"] = df["pm2.5cnc"].shift(2).bfill()
    df["pm10_lag1"] = df["pm10cnc"].shift(1).bfill()
    df["pm10_lag2"] = df["pm10cnc"].shift(2).bfill()

    # Month & Seasonal Encoding
    df["month"] = df.index.month

    def get_season(month):
        if month in [1, 2]:  
            return "Winter"
            return "Summer (Pre-Monsoon)"
        elif month in [6, 7, 8, 9]:  
            return "Monsoon"
        else:  
            return "Post-Monsoon (Autumn)"

    df["season"] = df["month"].apply(get_season)
    df["season_code"] = pd.Categorical(df["season"]).codes

    # Weekend Indicator
    # df["is_weekend"] = (df["day_of_week"] >= 5).astype(int)

    # Rate of Change Features
    # df["pm2.5_diff"] = df["pm2.5cnc"].diff().fillna(0)  # Avoid NaNs
    # df["pm10_diff"] = df["pm10cnc"].diff().fillna(0)  # Avoid NaNs

    # Drop Non-Numeric Columns
    df.drop(columns=["season"], inplace=True)
    # df.drop(columns=["hour"], inplace=True)
    # df.drop(columns=["minute"], inplace=True)
    

    # Final NaN Handling (Ensures No NaNs Remain)
    df.fillna(method="ffill", inplace=True)  # Forward-fill
    df.fillna(method="bfill", inplace=True)  # Back-fill

    return df


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ------------------ Encoding Functions ------------------

def encode_pm25(pm25):
    """
    Encodes PM2.5 value into a numeric score:
      1: Good (0–30 µg/m³)
      2: Satisfactory (31–60 µg/m³)
      3: Moderate (61–90 µg/m³)
      4: Poor (91–120 µg/m³)
      5: Very Poor (121–250 µg/m³)
      6: Severe (251 µg/m³ and above)
    """
    if pm25 <= 30:
        return 1
    elif pm25 <= 60:
        return 2
    elif pm25 <= 90:
        return 3
    elif pm25 <= 120:
        return 4
    elif pm25 <= 250:
        return 5
    else:
        return 6

def encode_pm10(pm10):
    """
    Encodes PM10 value into a numeric score:
      1: Good (0–50 µg/m³)
      2: Satisfactory (51–100 µg/m³)
      3: Moderately Polluted (101–250 µg/m³)
      4: Poor (251–350 µg/m³)
      5: Very Poor (351–430 µg/m³)
      6: Severe (431 µg/m³ and above)
    """
    if pm10 <= 50:
        return 1
    elif pm10 <= 100:
        return 2
    elif pm10 <= 250:
        return 3
    elif pm10 <= 350:
        return 4
    elif pm10 <= 430:
        return 5
    else:
        return 6

def combined_air_quality_label(pm25, pm10):
    """
    Combines PM2.5 and PM10 numeric scores by taking the floor of the average.
    For example, if one pollutant is 'Severe' (6) and the other 'Moderate' (3),
    the average (4.5) is floored to 4, which we interpret as 'Poor'.
    """
    score_pm25 = encode_pm25(pm25)
    score_pm10 = encode_pm10(pm10)
    # Floor the average using integer division (works for positive numbers)
    combined_score = (score_pm25 + score_pm10) // 2
    return combined_score

# Optional mapping back to text label if needed
combined_labels = {
    1: "Good",
    2: "Satisfactory",
    3: "Moderate",
    4: "Poor",
    5: "Very Poor",
    6: "Severe"
}

def process_air_quality_data(df):
    """
    Processes a DataFrame containing 'pm25' and 'pm10' columns:
      - Encodes each pollutant into its numeric score.
      - Creates a combined score.
      - Retains only numeric columns.
    """
    df['pm25_numeric'] = df['pm25'].apply(encode_pm25)
    df['pm10_numeric'] = df['pm10'].apply(encode_pm10)
    df['combined_numeric'] = df.apply(
        lambda row: combined_air_quality_label(row['pm25'], row['pm10']),
        axis=1
    )
    df['combined_label'] = df['combined_numeric'].map(combined_labels)
    
    # Keep only numeric columns (if desired)
    df_numeric = df.select_dtypes(include=[np.number])
    return df_numeric

# ------------------ Misclassification Identification ------------------

def check_misclassification(row, pollutant):
    """
    Identifies if the anomaly detection for a row is a misclassification.
    
    Logic:
      - Computes the combined numeric air quality label using both 'pm2.5cnc' and 'pm10cnc' columns.
      - If an anomaly is flagged (column value 1) but the combined label is in a milder category (1, 2, or 3),
        it is a misclassification.
      - If no anomaly is flagged (0) but the combined label is in a worse category (4, 5, or 6),
        it is a misclassification.
    
    Returns:
      - True if misclassification, False otherwise.
    """
    # Compute combined numeric label from the pollutant values (assumes both columns are available)
    combined = combined_air_quality_label(row["pm2.5cnc"], row["pm10cnc"])
    
    # Determine the anomaly flag column (e.g., "Anomaly_pm25cnc_features" for "pm2.5cnc")
    anomaly_col = f"Anomaly_{pollutant.replace('.', '')}"
    anomaly_flag = row[anomaly_col]
    
    if anomaly_flag == 1 and combined in [1, 2, 3]:
        return True
    # elif anomaly_flag == 0 and combined in [4, 5, 6]:
    #     return True
    else:
        return False

In [ ]:
def run_isoforest(train_data):
    """
    Trains an Isolation Forest on the given training data.
    Converts detected anomalies (-1) to normal (1) only if they fall within safe AQI ranges:
      - For 'pm2.5cnc': safe range is 0 to 90.
      - For 'pm10cnc': safe range is 0 to 350.
    Returns: (model, scores, labels)
    """
    # Train the model
    model = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
    model.fit(train_data)
    
    # Get outputs from Isolation Forest
    scores = model.decision_function(train_data)  # Higher => more normal
    labels = model.predict(train_data)              # 1 => normal, -1 => anomaly

    # Define safe range mask based on available pollutant columns
    # safe_range_mask = None
    # if 'pm2.5cnc' in train_data.columns and 'pm10cnc' in train_data.columns:
    #     mask_pm25 = (train_data["pm2.5cnc"] >= 0) & (train_data["pm2.5cnc"] <= 90)
    #     mask_pm10 = (train_data["pm10cnc"] >= 0) & (train_data["pm10cnc"] <= 350)
    #     safe_range_mask = mask_pm25 & mask_pm10
    # elif 'pm2.5cnc' in train_data.columns:
    #     safe_range_mask = (train_data["pm2.5cnc"] >= 0) & (train_data["pm2.5cnc"] <= 90)
    # elif 'pm10cnc' in train_data.columns:
    #     safe_range_mask = (train_data["pm10cnc"] >= 0) & (train_data["pm10cnc"] <= 350)
    
    # # For rows flagged as anomaly (-1) but within the safe range, force them to normal (1)
    # if safe_range_mask is not None:
    #     labels = np.where((labels == -1) & safe_range_mask, 1, labels)
    
    return model, scores, labels


def run_svm(train_data):
    """
    Trains a One-Class SVM on the given training data.
    Converts detected anomalies (-1) to normal (1) only if they fall within safe AQI ranges:
      - For 'pm2.5cnc': safe range is 0 to 90.
      - For 'pm10cnc': safe range is 0 to 350.
    Returns: (model, scores, labels)
    """
    # Train the model
    model = OneClassSVM(nu=0.05, kernel="rbf", gamma="scale")
    model.fit(train_data)
    
    # Get outputs from One-Class SVM
    scores = model.decision_function(train_data)  # Higher => more normal
    labels = model.predict(train_data)              # 1 => normal, -1 => anomaly

    # Define safe range mask based on available pollutant columns
    safe_range_mask = None
    if 'pm2.5cnc' in train_data.columns and 'pm10cnc' in train_data.columns:
        mask_pm25 = (train_data["pm2.5cnc"] >= 0) & (train_data["pm2.5cnc"] <= 250)
        mask_pm10 = (train_data["pm10cnc"] >= 0) & (train_data["pm10cnc"] <= 350)
        safe_range_mask = mask_pm25 & mask_pm10
    elif 'pm2.5cnc' in train_data.columns:
        safe_range_mask = (train_data["pm2.5cnc"] >= 0) & (train_data["pm2.5cnc"] <= 250)
    elif 'pm10cnc' in train_data.columns:
        safe_range_mask = (train_data["pm10cnc"] >= 0) & (train_data["pm10cnc"] <= 350)
    
    # For rows flagged as anomaly (-1) but within the safe range, force them to normal (1)
    if safe_range_mask is not None:
        labels = np.where((labels == -1) & safe_range_mask, 1, labels)
    
    return model, scores, labels

def split_data_random(df):
    """
    Splits the dataset into train (70%), test (20%), and validation (10%).

    Parameters:
    df (DataFrame): The dataset to split.

    Returns:
    train (DataFrame): 70% of data for training.
    test (DataFrame): 20% of data for testing.
    validation (DataFrame): 10% of data for final validation.
    """
    train, temp = train_test_split(df, test_size=0.3, random_state=42)  # 70% train, 30% temp
    test, validation = train_test_split(temp, test_size=1/3, random_state=42)  # 20% test, 10% validation
    return train, test, validation

def split_data(df):
    """
    Splits the dataset into train (70%), test (20%), and validation (10%) 
    in a historical order (earliest to latest).
    
    Parameters:
    df (DataFrame): The dataset to split. Assumes it is sorted by timestamp.
    
    Returns:
    train (DataFrame): First 70% of data for training.
    test (DataFrame): Next 20% of data for testing.
    validation (DataFrame): Final 10% of data for validation.
    """
    # Ensure data is sorted by timestamp (assuming a 'timestamp' column exists)
    df = df.sort_values(by='dt_time')

    # Compute split indices
    n = len(df)
    train_end = int(n * 0.7)
    test_end = int(n * 0.9)  # 70% train + 20% test = 90%, remaining 10% is validation

    # Split sequentially
    train = df.iloc[:train_end]
    test = df.iloc[train_end:test_end]
    validation = df.iloc[test_end:]

    return train, test, validation

In [ ]:
def plot_zoomed_anomalies(df, start_date, end_date, pollutant):
    # List possible column name formats for the anomaly column
    possible_names = [
        f"Anomaly_{pollutant}_features",
        f"Anomaly_{pollutant.replace('.', '')}_features",
        f"Anomaly_{pollutant}",
        f"Anomaly_{pollutant.replace('.', '')}"
    ]
    
    anomaly_col = None
    for name in possible_names:
        if name in df.columns:
            anomaly_col = name
            break

    if anomaly_col is None:
        raise KeyError(f"No anomaly column found for pollutant '{pollutant}'. Available columns: {df.columns}")
    
    # Filter DataFrame for the specified date range
    df_filtered = df.loc[start_date:end_date]
    
    plt.figure(figsize=(12, 6))
    plt.plot(df_filtered.index, df_filtered[pollutant], label=pollutant, color='blue')
    
    # Highlight anomalies using the found anomaly column
    anomalies = df_filtered[df_filtered[anomaly_col] == 1]
    plt.scatter(anomalies.index, anomalies[pollutant], color='red', label="Anomaly", marker='o')
    
    plt.xlabel("Date")
    plt.ylabel("Value")
    plt.title(f"Zoomed Anomalies for {pollutant}")
    plt.legend()
    plt.grid()
    plt.show()


In [1]:
def process_and_plot_anomalies(df, model_type, pollutant):
    """
    Generic anomaly detection pipeline for PM2.5 or PM10.

    Parameters:
      - df: DataFrame containing the air quality data.
      - model_type: One of {"svm", "iforest", "gmm", "dbscan", "lstm"}.
      - pollutant: Either "pm2.5cnc" or "pm10cnc".

    Returns:
      - Updated DataFrame with anomaly and misclassification labels.
    """
    
    # Ensure valid pollutant name
    if pollutant not in ["pm2.5cnc", "pm10cnc"]:
        raise ValueError("Invalid pollutant. Choose 'pm2.5cnc' or 'pm10cnc'.")
    
    # Define the anomaly column name (e.g., "Anomaly_pm25cnc")
    anomaly_col = f"Anomaly_{pollutant.replace('.', '')}"
    
    # Data Cleaning & Preparation
    df = clean_and_prepare(df)
    train, test, validation = split_data_random(df)
    
    # Select the correct model function
    model_functions = {
        "svm": run_svm,
        "iforest": run_isoforest,
        # Additional models can be added here as needed.
    }
    
    if model_type not in model_functions:
        raise ValueError("Invalid model_type. Choose from 'svm', 'iforest', 'gmm', 'dbscan', 'lstm'.")
    
    # Extract the pollutant column for anomaly detection
    train_data = train[[pollutant]]
    test_data = test[[pollutant]]
    val_data = validation[[pollutant]]
    
    # Run anomaly detection model on train, test, and validation splits
    model, train_scores, train_labels = model_functions[model_type](train_data)
    _, test_scores, test_labels = model_functions[model_type](test_data)
    _, val_scores, val_labels = model_functions[model_type](val_data)
    # print("Validation labels:", val_labels)
    
    # Store anomaly labels (1 = anomaly, 0 = normal)
    train["Anomaly"] = (train_labels == -1).astype(int)
    test["Anomaly"] = (test_labels == -1).astype(int)
    validation["Anomaly"] = (val_labels == -1).astype(int)
    
    # Merge anomaly labels back into the original DataFrame
    df[anomaly_col] = 0  # Default: no anomaly
    df.loc[train.index, anomaly_col] = train["Anomaly"]
    df.loc[test.index, anomaly_col] = test["Anomaly"]
    df.loc[validation.index, anomaly_col] = validation["Anomaly"]
    
    
    # Integrate misclassification identification
    misclass_col = f"Misclassification_{pollutant.replace('.', '')}"
    df[misclass_col] = df.apply(lambda row: check_misclassification(row, pollutant), axis=1)
    
    # # Create ground truth anomaly labels: rows with combinedlabels in [4, 5, 6] are true outliers
    # if 'combined_label' in df.columns:
    #     df["True_Anomaly"] = df["combined_label"].apply(lambda x: 1 if x in [4, 5, 6] else 0)
    # else:
    #     raise ValueError("DataFrame must contain 'combinedlabels' column for ground truth anomalies")
    
    # # Calculate metrics using the predicted labels from IsoForest and the ground truth
    # from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    # accuracy = accuracy_score(df["True_Anomaly"], df["Anomaly_IsoForest"])
    # precision = precision_score(df["True_Anomaly"], df["Anomaly_IsoForest"])
    # recall = recall_score(df["True_Anomaly"], df["Anomaly_IsoForest"])
    # f1 = f1_score(df["True_Anomaly"], df["Anomaly_IsoForest"])
    
    # print("Accuracy:", accuracy)
    # print("Precision:", precision)
    # print("Recall:", recall)
    # print("F1 Score:", f1)
    
    # ------------------ Plotting ------------------
    import matplotlib.pyplot as plt
    plt.figure(figsize=(12, 5))
    plt.scatter(train.index, train[pollutant], c=train["Anomaly"], cmap="coolwarm", label="Train", s=15)
    # You can uncomment these lines if you wish to plot test and validation splits as well.
    # plt.scatter(test.index, test[pollutant], c=test["Anomaly"], cmap="coolwarm", label="Test", s=15, marker="x")
    # plt.scatter(validation.index, validation[pollutant], c=validation["Anomaly"], cmap="coolwarm", label="Validation", s=15, marker="^")
    plt.title(f"{pollutant.upper()} Concentration Anomalies ({model_type.upper()})")
    plt.xlabel("Time")
    plt.ylabel(f"{pollutant} Concentration")
    plt.legend()
    plt.show()
    
    # Print anomaly counts for each split
    print(f"Train anomalies ({pollutant}): {train['Anomaly'].sum()}")
    print(f"Test anomalies ({pollutant}): {test['Anomaly'].sum()}")
    print(f"Validation anomalies ({pollutant}): {validation['Anomaly'].sum()}")
    
    # Print misclassification count
    total_misclassified = df[misclass_col].sum()
    print(f"Total misclassifications for {pollutant}: {total_misclassified}")
    
    return df


In [2]:
df = pd.read_csv("site_104_data.csv", parse_dates=["dt_time"])

NameError: name 'pd' is not defined

In [ ]:
# df25svm = process_and_plot_anomalies(df, model_type="svm", pollutant="pm2.5cnc")
# df10svm = process_and_plot_anomalies(df,model_type = "svm",pollutant="pm10cnc")
df10iforest = process_and_plot_anomalies(df, model_type="iforest", pollutant="pm10cnc")
df25iforest = process_and_plot_anomalies(df,model_type="iforest",pollutant="pm2.5cnc")

In [ ]:
df25iforest

In [ ]:
merged_df = pd.merge(
    df10iforest[['pm10cnc', 'Anomaly_pm10cnc']], 
    df25iforest[['pm2.5cnc', 'Anomaly_pm25cnc']], 
    left_index=True, 
    right_index=True, 
    how="inner"
)

# Reset index if necessary
merged_df.reset_index(inplace=True)

# Display the final dataframe
print(merged_df.head())


In [ ]:
import pandas as pd
import json
import os

# Function to generate anomaly JSON data
def get_anomaly_data(df25, df10, site_id):
    # Drop unnecessary columns
    df10 = df10.drop(columns=["index", "level_0"], errors="ignore")
    df25 = df25.drop(columns=["index", "level_0"], errors="ignore")

    # Set 'dt_time' as index
    df10.set_index("dt_time", inplace=True)
    df25.set_index("dt_time", inplace=True)

    # Merge both DataFrames on 'dt_time' (datetime index)
    merged_df = pd.merge(df10, df25, on="dt_time", how="inner")

    # Reset index for JSON output & convert dt_time to string
    merged_df.reset_index(inplace=True)
    merged_df["dt_time"] = merged_df["dt_time"].astype(str)

    # Return JSON formatted data for given site_id
    return {site_id: merged_df.to_dict(orient="records")}

# Function to save JSON data without overwriting existing keys
def jsonify(data, filename="output.json"):
    # Load existing data if file exists
    if os.path.exists(filename):
        with open(filename, "r") as f:
            existing_data = json.load(f)
    else:
        existing_data = {}

    # Update with new site_id data without deleting old ones
    existing_data.update(data)

    # Save updated JSON
    with open(filename, "w") as f:
        json.dump(existing_data, f, indent=4)  # Pretty-print JSON

# Example usage: Add new data for site_id 104 while keeping 117
jsonify(get_anomaly_data(df25iforest, df10iforest, 104))

In [ ]:
df10iforest

In [ ]:
# def process_and_plot_anomalies(df, model_type, pollutant):
#     """
#     Generic anomaly detection pipeline for PM2.5 or PM10.

#     Parameters:
#     - df: DataFrame containing the air quality data.
#     - model_type: One of {"svm", "iforest", "gmm", "dbscan", "lstm"}.
#     - pollutant: Either "pm2.5cnc" or "pm10cnc".

#     Returns:
#     - Updated DataFrame with anomaly labels.
#     """

#     # Ensure valid pollutant name
#     if pollutant not in ["pm2.5cnc", "pm10cnc"]:
#         raise ValueError("Invalid pollutant. Choose 'pm2.5cnc' or 'pm10cnc'.")

#     # Mapping of pollutant to its anomaly column
#     anomaly_col = f"Anomaly_{pollutant.replace('.', '')}"

#     # Data Cleaning & Preparation
#     df = clean_and_prepare(df)
#     train, test, validation = split_data(df)

#     # Select the correct model function
#     model_functions = {
#         "svm": run_svm,
#         "iforest": run_isoforest,
#     }

#     if model_type not in model_functions:
#         raise ValueError("Invalid model_type. Choose from 'svm', 'iforest', 'gmm', 'dbscan', 'lstm'.")

#     # Extract only the pollutant column for anomaly detection
#     train_data = train[[pollutant]]
#     test_data = test[[pollutant]]
#     val_data = validation[[pollutant]]

#     # Run anomaly detection model
#     model, train_scores, train_labels = model_functions[model_type](train_data)
#     _, test_scores, test_labels = model_functions[model_type](test_data)
#     _, val_scores, val_labels = model_functions[model_type](val_data)

#     # Store anomaly labels
#     train["Anomaly"] = (train_labels == -1).astype(int)
#     test["Anomaly"] = (test_labels == -1).astype(int)
#     validation["Anomaly"] = (val_labels == -1).astype(int)

#     # Merge back into the original dataframe
#     df[anomaly_col] = 0  # Default: no anomaly
#     df.loc[train.index, anomaly_col] = train["Anomaly"]
#     df.loc[test.index, anomaly_col] = test["Anomaly"]
#     df.loc[validation.index, anomaly_col] = validation["Anomaly"]

#     # Plot anomalies
#     plt.figure(figsize=(12, 5))
#     plt.scatter(train.index, train[pollutant], c=train["Anomaly"], cmap="coolwarm", label="Train", s=15)
#     plt.scatter(test.index, test[pollutant], c=test["Anomaly"], cmap="coolwarm", label="Test", s=15, marker="x")
#     plt.scatter(validation.index, validation[pollutant], c=validation["Anomaly"], cmap="coolwarm", label="Validation", s=15, marker="^")
#     plt.title(f"{pollutant.upper()} Concentration Anomalies ({model_type.upper()})")
#     plt.xlabel("Time")
#     plt.ylabel(f"{pollutant} Concentration")
#     plt.legend()
#     plt.show()

#     # Print anomaly counts
#     print(f"Train anomalies ({pollutant}): {train['Anomaly'].sum()}")
#     print(f"Test anomalies ({pollutant}): {test['Anomaly'].sum()}")
#     print(f"Validation anomalies ({pollutant}): {validation['Anomaly'].sum()}")

#     return df


In [ ]:

# def check_misclassification_single(row, pollutant):
#     """
#     Identifies if the anomaly detection for a given row is a misclassification.
    
#     Logic for a single pollutant:
#       - For PM2.5:
#           * If anomaly flagged (==1) but value encodes to 1-3 (mild), then misclassified.
#           * If no anomaly (==0) but value encodes to 4-6 (severe), then misclassified.
#       - For PM10:
#           * If anomaly flagged (==1) but value encodes to 1-3 (mild), then misclassified.
#           * If no anomaly (==0) but value encodes to 4-6 (severe), then misclassified.
#     """
#     # Determine encoded severity based on pollutant type.
#     if pollutant == "pm2.5cnc":
#         encoded = encode_pm25(row[pollutant])
#     elif pollutant == "pm10cnc":
#         encoded = encode_pm10(row[pollutant])
#     else:
#         raise ValueError("Invalid pollutant. Choose 'pm2.5cnc' or 'pm10cnc'.")
    
#     # Define ranges: 1-3 (mild) and 4-6 (severe)
#     is_anomaly = row[f"Anomaly_{pollutant.replace('.', '')}"]
#     if is_anomaly == 1 and encoded in [1, 2, 3]:
#         return True
#     # elif is_anomaly == 0 and encoded in [4, 5, 6]:
#     #     return True
#     else:
#         return False

# # ------------------ Anomaly Detection Pipeline ------------------

In [ ]:
# def process_and_plot_with_features(df, model_type, pollutant):
#     """
#     Anomaly detection pipeline using additional features from clean_and_prepare.
    
#     Parameters:
#       - df: DataFrame containing the air quality data.
#       - model_type: One of {"svm", "iforest", "gmm", "dbscan", "lstm"}.
#       - pollutant: Either "pm2.5cnc" or "pm10cnc".
      
#     Returns:
#       - Updated DataFrame with anomaly and misclassification labels.
#     """
    
#     # Ensure valid pollutant selection
#     if pollutant not in ["pm2.5cnc", "pm10cnc"]:
#         raise ValueError("Invalid pollutant. Choose 'pm2.5cnc' or 'pm10cnc'.")
    
#     # Construct the anomaly column name (remove '.' from pollutant name)
#     anomaly_col = f"Anomaly_{pollutant.replace('.', '')}_features"
    
#     # Apply feature engineering (assumed to be defined elsewhere)
#     df = clean_and_prepare(df)
    
#     # Train-Test-Validation Split (assumed to be defined elsewhere)
#     train, test, validation = split_data(df)
    
#     # Select features: all numerical columns except timestamp and the pollutant itself
#     feature_columns = [col for col in df.columns if col not in ["dt_time", pollutant]]
    
#     # Extract feature matrices
#     train_X, test_X, val_X = train[feature_columns], test[feature_columns], validation[feature_columns]
    
#     # Define available model functions (assumed to be defined elsewhere)
#     model_functions = {
#         "svm": run_svm,
#         "iforest": run_isoforest,
#         # "gmm": run_gmm,
#         # "dbscan": run_dbscan,
#         # "lstm": run_lstm_autoencoder
#     }
    
#     if model_type not in model_functions:
#         raise ValueError("Invalid model_type. Choose from 'svm', 'iforest', 'gmm', 'dbscan', 'lstm'.")
    
#     # Run the anomaly detection model on train, test, and validation splits
#     model, train_scores, train_labels = model_functions[model_type](train_X)
#     _, test_scores, test_labels = model_functions[model_type](test_X)
#     _, val_scores, val_labels = model_functions[model_type](val_X)
    
#     # Convert labels to binary (1 = anomaly, 0 = normal)
#     train["Anomaly"] = (train_labels == -1).astype(int)
#     test["Anomaly"] = (test_labels == -1).astype(int)
#     validation["Anomaly"] = (val_labels == -1).astype(int)
    
#     # Merge anomaly results back into the original DataFrame
#     df[anomaly_col] = 0  # Default: no anomaly
#     df.loc[train.index, anomaly_col] = train["Anomaly"]
#     df.loc[test.index, anomaly_col] = test["Anomaly"]
#     df.loc[validation.index, anomaly_col] = validation["Anomaly"]
    
#     # --- Integrate Misclassification Identification ---
#     # Add a new column to mark misclassifications based on the logic provided.
#     misclass_col = f"Misclassification_{pollutant.replace('.', '')}_features"
#     df[misclass_col] = df.apply(lambda row: check_misclassification(row, pollutant), axis=1)
    
#     # ------------------ Plotting ------------------
#     plt.figure(figsize=(12, 5))
#     plt.scatter(train.index, train[pollutant], c=train["Anomaly"], cmap="coolwarm", label="Train", s=15)
#     plt.scatter(test.index, test[pollutant], c=test["Anomaly"], cmap="coolwarm", label="Test", s=15, marker="x")
#     plt.scatter(validation.index, validation[pollutant], c=validation["Anomaly"], cmap="coolwarm", label="Validation", s=15, marker="^")
#     plt.title(f"{pollutant.upper()} Concentration Anomalies with Features ({model_type.upper()})")
#     plt.xlabel("Time")
#     plt.ylabel(f"{pollutant} Concentration")
#     plt.legend()
#     plt.show()
    
#     # Print anomaly counts
#     print(f"Train anomalies ({pollutant} + features): {train['Anomaly'].sum()}")
#     print(f"Test anomalies ({pollutant} + features): {test['Anomaly'].sum()}")
#     print(f"Validation anomalies ({pollutant} + features): {validation['Anomaly'].sum()}")
    
#     # Print misclassification counts
#     total_misclassified = df[misclass_col].sum()
#     print(f"Total misclassifications for {pollutant} with features: {total_misclassified}")
    
#     return df

In [ ]:
# numeric_cols = df.select_dtypes(include=[np.number]).columns
# X = df[numeric_cols].values

# # Standardize features
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # Apply PCA with all components to see full variance structure
# pca = PCA(n_components=X_scaled.shape[1])
# pca.fit(X_scaled)
# explained_variance = pca.explained_variance_ratio_
# cum_explained_variance = np.cumsum(explained_variance)

# # ------------------ Plot Cumulative Explained Variance (Elbow Plot) ------------------

# plt.figure(figsize=(8,6))
# plt.plot(np.arange(1, len(cum_explained_variance) + 1), cum_explained_variance, marker='o', linestyle='--')
# plt.xlabel("Number of Components")
# plt.ylabel("Cumulative Explained Variance")
# plt.title("Cumulative Explained Variance vs. Number of PCA Components")
# plt.axhline(y=0.90, color='red', linestyle='-')
# plt.text(1, 0.85, "90% Threshold", color='red', fontsize=12)
# plt.grid(True)
# plt.show()

# # ------------------ Plot Scree Plot ------------------

# plt.figure(figsize=(8,6))
# plt.bar(np.arange(1, len(explained_variance) + 1), explained_variance, color='skyblue')
# plt.xlabel("Principal Component")
# plt.ylabel("Explained Variance Ratio")
# plt.title("Scree Plot")
# plt.grid(True)
# plt.show()

# # ------------------ Print Out Variance and Feature Loadings ------------------

# print("Cumulative Explained Variance by Component:")
# for i, cum_var in enumerate(cum_explained_variance, start=1):
#     print(f"Component {i}: {cum_var:.2f}")

# # Create a DataFrame for loadings to inspect which features contribute most
# loadings = pca.components_.T
# loading_df = pd.DataFrame(loadings, index=numeric_cols,
#                           columns=[f"PC{i}" for i in range(1, len(explained_variance)+1)])
# print("\nFeature Loadings for each Principal Component:")
# print(loading_df)

# # Optionally, you can visually inspect loadings for the first few components:
# # For example, a bar plot for PC1 loadings:
# plt.figure(figsize=(10, 6))
# pc1_loadings = loading_df["PC1"].abs().sort_values(ascending=False)
# pc1_loadings.plot(kind='bar', color='green')
# plt.xlabel("Features")
# plt.ylabel("Absolute Loading on PC1")
# plt.title("Feature Contribution to PC1")
# plt.show()